# Reinforcement Learning

## Preliminary

In [1]:
%%bash
pip3 install 'gymnasium[all]' ipywidgets jupyter_contrib_nbextensions

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-7)


Defaulting to user installation because normal site-packages is not writeable


In [14]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm 


from IPython import display

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [16]:
%matplotlib inline

## Discrete Action Spacce

In [17]:
ENV = "LunarLander-v2"

env = gym.make(ENV, continuous=False)

In [18]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [19]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [20]:
episode_durations = []
def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [21]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [28]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

KeyboardInterrupt: 

In [23]:
import ipywidgets as widgets
from PIL import Image
import time
import io

class ArrayRenderWidget(widgets.Image):
    def render(self, frame):
        img = Image.fromarray(frame, 'RGB')
        with io.BytesIO() as buf:
            img.save(buf, format='PNG')
            image_data = buf.getvalue()
        self.value = image_data
        
image_widget = ArrayRenderWidget(
    format='png',
    width=600,
    height=400,
)

In [27]:
env = gym.make(ENV, render_mode="rgb_array")

observation, info = env.reset()

display.display(image_widget)
image_widget.render(env.render())
for _ in range(1000):
    image_widget.render(env.render())


    # action = env.action_space.sample()
    rewards = policy_net(torch.tensor(observation).to(device))
    action = torch.argmax(rewards)
    action = action.detach().cpu().numpy()
    
    observation, reward, terminated, truncated, info = env.step(action)
    time.sleep(1/30)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

ArrayRenderWidget(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01\x90\x08\x02\x00\x00\x00…

KeyboardInterrupt: 

## Continuos Action Space

### Discretization

In [69]:
ENV = "LunarLander-v2"

env = gym.make(ENV, continuous=True)

In [5]:
NUM_STEPS = 10

action_high = torch.tensor(env.action_space.high, device=device)
action_low = torch.tensor(env.action_space.low, device=device)

def discretized_to_continuous(x, one_hot=False):
    if one_hot:
        x = (x==torch.max(x)).nonzero()[0]
    return x / (NUM_STEPS - 1) * (action_high - action_low) + action_low

def continuous_to_discretized(x, one_hot=False):
    disc = (x - action_low) / (action_high - action_low) * (NUM_STEPS - 1)
    if one_hot:
        # mask = torch.nn.functional.one_hot(disc.type(torch.LongTensor), num_classes=NUM_STEPS)
        mask = torch.zeros([NUM_STEPS for _ in range(env.action_space.shape[0])], device=device)
        disc = disc.type(torch.LongTensor)
        mask[torch.chunk(disc, len(disc))] = 1
        disc = mask
    return disc

sample = torch.randn([NUM_STEPS for _ in range(env.action_space.shape[0])], device=device)
# sample = (torch.rand(env.action_space.shape[0]) * NUM_STEPS).type(torch.LongTensor).to(device=device)
cont = discretized_to_continuous(sample, one_hot=True)
disc =  continuous_to_discretized(cont)
one_hot = continuous_to_discretized(cont, True)

print(sample)
print(cont)
print(disc)
print(one_hot)

tensor([[-2.3603e-01,  2.1152e+00,  5.1382e-01, -7.5329e-01, -5.0548e-01,
          1.0356e+00,  9.1408e-01, -1.8720e+00, -4.6602e-01, -8.5839e-01],
        [-1.1753e-02, -2.1365e+00, -2.4842e+00, -7.4930e-01,  2.0107e-01,
          5.1604e-01, -2.1930e-01,  3.8726e-01, -6.8372e-01, -8.0837e-02],
        [-5.8296e-01,  7.5303e-01, -1.8795e-01,  7.7095e-02,  1.0578e-01,
          6.4353e-01, -9.7231e-01, -1.1102e+00,  9.3758e-02, -2.9694e-01],
        [-1.4749e+00,  1.2497e+00, -1.6271e+00,  1.8139e+00, -4.9072e-01,
         -2.3762e-01,  6.4897e-01, -2.8345e-01,  5.4384e-01, -4.4040e-01],
        [ 7.5439e-01,  1.3158e+00,  4.6082e-01, -7.8481e-01, -1.2086e+00,
          3.7185e-01, -3.0356e-01,  1.1270e+00, -4.7545e-01, -1.2626e+00],
        [ 1.0954e+00, -4.7233e-01,  5.8889e-01, -2.4507e+00,  1.2598e+00,
          1.5547e+00, -2.6286e-01, -5.3695e-01, -8.6354e-01, -3.9769e-02],
        [ 2.7810e+00,  1.1284e+00,  1.0309e+00,  1.4198e+00,  5.9727e-01,
          8.0605e-01,  4.4906e-0

In [6]:
class DQN(nn.Module):
    def __init__(self, n_observations, action_space_dim, num_steps):
        super(DQN, self).__init__()
        self.action_space_dim = action_space_dim
        self.num_steps = num_steps
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)

        num_outs = num_steps ** action_space_dim
                            
        self.layer3 = nn.Linear(128, num_outs)

    def forward(self, x):
        is_batch = len(x.shape) == 2
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        if is_batch:
            x = x.reshape([x.shape[0], ] + [self.num_steps for _ in range(self.action_space_dim)])
        else: 
            x = x.reshape([self.num_steps for _ in range(self.action_space_dim)])

        return x

# Get number of actions from gym action space
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)
action_space_dim = env.action_space.shape[0]

policy_net = DQN(n_observations, action_space_dim, NUM_STEPS).to(device)
target_net = DQN(n_observations, action_space_dim, NUM_STEPS).to(device)
target_net.load_state_dict(policy_net.state_dict())

<All keys matched successfully>

In [7]:
test_state_batch = torch.zeros((128, *env.observation_space.shape), device=device)
out = policy_net(test_state_batch)
out.flatten(start_dim=1).shape

torch.Size([128, 100])

In [70]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.01
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)

def select_action(state, step):
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * step / EPS_DECAY)
    if sample > eps_threshold:
        with torch.no_grad():
            return discretized_to_continuous(policy_net(state), one_hot=True)
    else:
        return torch.tensor(env.action_space.sample(), device=device)

In [71]:
observation, _ = env.reset()
action = select_action(torch.tensor(observation, device=device), 10000).detach().cpu().numpy()
print(action)

observation, reward, *_ = env.step(action)
reward

[ 1.        -0.1111111]


-4.501729208911217

In [72]:
def discrete_to_flat_index(x):
    x *= torch.tensor([NUM_STEPS, 0.0], device=device) 
    return torch.sum(x).type(torch.LongTensor)
    

In [77]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return torch.tensor(0)
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.stack([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.stack(batch.state)
    action_batch = torch.stack([discrete_to_flat_index(continuous_to_discretized(action)) for action in batch.action]).type(torch.LongTensor).to(device)
    reward_batch = torch.cat(batch.reward)
    
    state_action_values = policy_net(state_batch).flatten(start_dim=1).gather(1, action_batch.unsqueeze(0)).squeeze()
    
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).flatten(start_dim=1).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values)
     
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()
    return loss

In [78]:
optimize_model()

tensor(0)

In [79]:
def plot(rewards, losses, show_result=False):
    plt.figure(1)
    losses_t = torch.tensor(losses, dtype=torch.float)
    rewards_t = torch.tensor(rewards, dtype=torch.float)

    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.plot(losses_t.numpy(), label="loss")
    # plt.plot(rewards_t.numpy(), label="reward")
    plt.legend()

    plt.pause(0.001)  # pause a bit so that plots are updated
    if not show_result:
        display.display(plt.gcf())
        display.clear_output(wait=True)
    else:
        display.display(plt.gcf())

In [80]:
num_episodes = 2000
episode_durations = []
rewards = []
losses = []
step = 0

# for i_episode in tqdm(range(num_episodes)):
for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device)
    episode_reward = 0
    episode_losses = []

    for t in count():
        action = select_action(state, step)
        observation, reward, terminated, truncated, _ = env.step(action.detach().cpu().numpy())
        reward = torch.tensor([reward], device=device)
        episode_reward += reward
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        episode_losses.append(loss)

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            rewards.append(episode_reward)
            losses.append(torch.mean(torch.tensor(episode_losses)))
            print("Rewards: ", rewards[-1], " Losses: ", losses[-1])
            plot(rewards, losses)
            break

<Figure size 640x480 with 0 Axes>

In [68]:
env = gym.make(ENV, continuous=True, render_mode="rgb_array")

observation, info = env.reset()

display.display(image_widget)

for _ in range(1000):
    image_widget.render(env.render())
    # action = env.action_space.sample()
    action = discretized_to_continuous(policy_net(torch.tensor(observation).to(device)), one_hot=True)
    action = action.detach().cpu().numpy()
    
    observation, reward, terminated, truncated, info = env.step(action)
    time.sleep(1/30)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

ArrayRenderWidget(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01\x90\x08\x02\x00\x00\x00…

KeyboardInterrupt: 